# EcoPoints Data Analysis

## Import needed libraries

In [1]:
import json
import pandas as pd
import pydeck as pdk
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

from dash import Dash, dcc, html, Input, Output

In [2]:
with open('data.json') as f:
    data = json.load(f)

## Data Preperation

In [3]:
timestamps = []
engine_load = []
engine_rpm = []
coolant_temp = []
altitude = []
gps_speed = []
vehicle_speed = []
armin_speed = []
latitude = []
longitude = []

for entry in data:
    timestamps.append(pd.to_datetime(entry.get('timestamp'), unit='ms'))
    
    entry_data = entry.get('data', {})
    engine_load.append(float(entry_data.get('Engine Load', '0').replace(',', '.')))
    engine_rpm.append(float(entry_data.get('Engine RPM', '0').replace(',', '.')))
    coolant_temp.append(float(entry_data.get('Engine Coolant Temperature', '0').replace(',', '.')))
    altitude.append(float(entry_data.get('Altitude', '0').replace(',', '.')))
    gps_speed.append(float(entry_data.get('Gps-Speed', '0').replace(',', '.')))
    vehicle_speed.append(float(entry_data.get('Vehicle Speed', '0').replace(',', '.')))
    armin_speed.append(float(entry_data.get('Armin-Speed', '0').replace(',', '.')))
    latitude.append(float(entry_data.get('Latitude', '0')))
    longitude.append(float(entry_data.get('Longitude', '0')))

df = pd.DataFrame({
    'timestamp': timestamps,
    'Engine Load': engine_load,
    'Engine RPM': engine_rpm,
    'Coolant Temperature': coolant_temp,
    'Altitude': altitude,
    'Gps Speed': gps_speed,
    'Vehicle Speed': vehicle_speed,
    'Armin Speed': armin_speed,
    'Latitude': latitude,
    'Longitude': longitude
})

df_normalized = df.copy() 
columns_to_normalize = ['Engine Load', 'Engine RPM', 'Coolant Temperature', 'Altitude', 'Gps Speed', 'Vehicle Speed', 'Armin Speed']

for col in columns_to_normalize:
    df_normalized[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())


In [4]:
df = df[:-1]  
df_normalized = df_normalized[:-1]

In [5]:
print(df.head())

                timestamp  Engine Load  Engine RPM  Coolant Temperature  \
0 2024-10-24 13:21:13.199         78.0      1764.0                 98.0   
1 2024-10-24 13:21:16.865         79.2      2019.0                 98.0   
2 2024-10-24 13:21:20.531         64.3      2542.0                 97.0   
3 2024-10-24 13:21:24.177          0.0      1716.0                 97.0   
4 2024-10-24 13:21:27.816         66.3      1532.0                 97.0   

     Altitude  Gps Speed  Vehicle Speed  Armin Speed   Latitude  Longitude  
0  533.262937       45.0           41.0         44.0  48.322290  14.256009  
1  537.843367       56.0           50.0         58.0  48.322815  14.255925  
2  542.902183       60.0           61.0         60.0  48.323429  14.255708  
3  545.912065       55.0           57.0         56.0  48.323844  14.255458  
4  548.228076       52.0           51.0         49.0  48.324335  14.255235  


In [6]:
print(df_normalized.head())

                timestamp  Engine Load  Engine RPM  Coolant Temperature  \
0 2024-10-24 13:21:13.199        0.780    0.669450             1.000000   
1 2024-10-24 13:21:16.865        0.792    0.766224             1.000000   
2 2024-10-24 13:21:20.531        0.643    0.964706             0.989796   
3 2024-10-24 13:21:24.177        0.000    0.651233             0.989796   
4 2024-10-24 13:21:27.816        0.663    0.581404             0.989796   

   Altitude  Gps Speed  Vehicle Speed  Armin Speed   Latitude  Longitude  
0  0.801975   0.463918       0.440860     0.435644  48.322290  14.256009  
1  0.808864   0.577320       0.537634     0.574257  48.322815  14.255925  
2  0.816472   0.618557       0.655914     0.594059  48.323429  14.255708  
3  0.820998   0.567010       0.612903     0.554455  48.323844  14.255458  
4  0.824482   0.536082       0.548387     0.485149  48.324335  14.255235  


## Driven Path

In [7]:
path_data = [
    {"path": [[row['Longitude'], row['Latitude']] for _, row in df.iterrows()]} 
]
layer = pdk.Layer(
    "PathLayer",
    path_data,
    get_path='path',
    get_width=1,  
    get_color='[0, 128, 255]',  
    width_scale=10,
    pickable=True,
)
view_state = pdk.ViewState(
    latitude=df['Latitude'].mean() if not df['Latitude'].empty else 0.0, 
    longitude=df['Longitude'].mean() if not df['Longitude'].empty else 0.0, 
    zoom=12, 
    bearing=0,
    pitch=0,
)

deck = pdk.Deck(layers=[layer], initial_view_state=view_state)
deck.to_html("driven_route_map.html")

In [8]:
path_data = df[['Longitude', 'Latitude']].values.tolist()

path_layer = pdk.Layer(
    "PathLayer",
    data=[path_data],
    get_path='path',  
    get_color='[255, 0, 0]',  
    width_scale=10,  
    width_min_pixels=1,  
    get_width=1  
)

# Define the column layer for elevation
column_layer = pdk.Layer(
    "ColumnLayer",
    df,
    get_position='[Longitude, Latitude]',
    get_elevation='Altitude',
    elevation_scale=0.1, 
    radius=5,
    get_fill_color='[255, 0, 0]',
    pickable=True,
)

view_state = pdk.ViewState(
    latitude=df['Latitude'].mean(),
    longitude=df['Longitude'].mean(),
    zoom=12,
    bearing=0,
    pitch=50,
)

deck = pdk.Deck(layers=[path_layer, column_layer], initial_view_state=view_state)
deck.to_html("3d_path_map_with_lines_and_elevation.html")

## All Car Metrics in one

### Original Values

In [9]:
carMetrOrig = Dash(__name__)

carMetrOrig.layout = html.Div([
    html.H4('Vehicle Data Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'}),
    dcc.Checklist(
        id="checklist",
        options=[
            {'label': 'Engine Load', 'value': 'Engine Load'},
            {'label': 'Engine RPM', 'value': 'Engine RPM'},
            {'label': 'Coolant Temperature', 'value': 'Coolant Temperature'},
            {'label': 'Altitude', 'value': 'Altitude'},
            {'label': 'Gps Speed', 'value': 'Gps Speed'},
            {'label': 'Vehicle Speed', 'value': 'Vehicle Speed'},
            {'label': 'Armin Speed', 'value': 'Armin Speed'}
        ],
        value=['Engine RPM'],
        inline=True
    ),
])

@carMetrOrig.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(selected_metrics):
    fig = px.line(df, x='timestamp', y=selected_metrics)
    
    fig.update_layout(
        title="Selected Vehicle Metrics Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Value",
        legend_title="Metrics",
        xaxis={
            'rangeslider': {'visible': True},
            'type': 'date', 
            'range': [df['timestamp'].min(), df['timestamp'].max()]
        }
    )
    return fig

if __name__ == '__main__':
    carMetrOrig.run_server(debug=True, port=8050)


### Normalized Values

In [10]:
carMetrNorm = Dash(__name__)

carMetrNorm.layout = html.Div([
    html.H4('Normalized Vehicle Data Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'}),
    dcc.Checklist(
        id="checklist",
        options=[
            {'label': 'No. Engine Load', 'value': 'Engine Load'},
            {'label': 'No. Engine RPM', 'value': 'Engine RPM'},
            {'label': 'No. Coolant Temperature', 'value': 'Coolant Temperature'},
            {'label': 'No. Altitude', 'value': 'Altitude'},
            {'label': 'No. Gps Speed', 'value': 'Gps Speed'},
            {'label': 'No. Vehicle Speed', 'value': 'Vehicle Speed'},
            {'label': 'No. Armin Speed', 'value': 'Armin Speed'}
        ],
        value=['Engine RPM'],
        inline=True
    ),
])

@carMetrNorm.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(selected_metrics):
    fig = px.line(df_normalized, x='timestamp', y=selected_metrics)
    
    fig.update_layout(
        title="Selected Normalized Vehicle Metrics Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Normalized Value",
        legend_title="Metrics",
        xaxis={
            'rangeslider': {'visible': True},
            'type': 'date', 
            'range': [df['timestamp'].min(), df['timestamp'].max()] 
        }
    )
    return fig

if __name__ == '__main__':
    carMetrNorm.run_server(debug=True, port=8051)

## Altitude and Engine Load (Normalized)

In [11]:
df['Normalized Altitude'] = (df['Altitude'] - df['Altitude'].min()) / (df['Altitude'].max() - df['Altitude'].min())
df['Normalized Engine Load'] = (df['Engine Load'] - df['Engine Load'].min()) / (df['Engine Load'].max() - df['Engine Load'].min())

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Normalized Altitude and Engine Load Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'})
])

@app.callback(
    Output("graph", "figure"), 
    Input("graph", "id") 
)
def update_line_chart(_):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['Normalized Altitude'],
        mode='lines',
        name='Normalized Altitude',
        line=dict(color='orange', width=2),
        marker=dict(size=5, symbol='circle'),
    ))

    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['Normalized Engine Load'],
        mode='lines',
        name='Normalized Engine Load',
        line=dict(color='blue', width=2),
        marker=dict(size=5, symbol='circle'),
    ))

    fig.update_layout(
        title='Normalized Altitude and Engine Load Over Time',
        xaxis_title='Timestamp',
        yaxis_title='Normalized Values',
        legend_title='Legend',
        xaxis=dict(
            showgrid=True,
            rangeslider=dict(visible=True),
            type='date',
            range=[df['timestamp'].min(), df['timestamp'].max()]
        ),
        yaxis=dict(showgrid=True),
        hovermode='x unified'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)